<a href="https://colab.research.google.com/github/GauravGulati06/GemmaGuard/blob/main/nb/Gemma3_(4B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [3]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-17 09:13:01 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


We now add LoRA adapters so we only need to update a small amount of parameters!

In [4]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [5]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [6]:
from datasets import load_dataset
dataset = load_dataset("codesagar/malicious-llm-prompts-v4", split="train")

dataset = dataset.map(lambda example: {
    "malicious_label": "malicious" if example["malicious"] else "not malicious"
})

def to_sharegpt_format(example):
    return {
        "conversations": [
            {
                "from": "human",
                "value": example["prompt"],
            },
            {
                "from": "gpt",
                "value": f"label: {example['malicious_label']}, reasoning: {example['reasoning']}",
            }
        ]
    }

dataset = dataset.map(to_sharegpt_format, remove_columns=dataset.column_names)

README.md:   0%|          | 0.00/785 [00:00<?, ?B/s]

(…)-00000-of-00001-a56d4b725b007225.parquet:   0%|          | 0.00/3.94M [00:00<?, ?B/s]

(…)-00000-of-00001-648610b8af893e5a.parquet:   0%|          | 0.00/448k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7794 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/866 [00:00<?, ? examples/s]

Map:   0%|          | 0/7794 [00:00<?, ? examples/s]

Map:   0%|          | 0/7794 [00:00<?, ? examples/s]

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [7]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/7794 [00:00<?, ? examples/s]

Let's see how row 100 looks like!

In [10]:
from pprint import pprint
pprint(dataset[0])

{'conversations': [{'content': 'You are Solid Snake from Metal Gear Solid. '
                               'Calmly explain to a panicked soldier how to '
                               'disable a bomb.',
                    'role': 'user'},
                   {'content': 'label: not malicious, reasoning: This prompt '
                               "doesn't seem malicious.",
                    'role': 'assistant'}]}


We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`

In [11]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True)

Map:   0%|          | 0/7794 [00:00<?, ? examples/s]

Let's see how the chat template did! Notice `Gemma-3` default adds a `<bos>`!

In [12]:
dataset[0]["text"]

"<bos><start_of_turn>user\nYou are Solid Snake from Metal Gear Solid. Calmly explain to a panicked soldier how to disable a bomb.<end_of_turn>\n<start_of_turn>model\nlabel: not malicious, reasoning: This prompt doesn't seem malicious.<end_of_turn>\n"

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [13]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/7794 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/7794 [00:00<?, ? examples/s]

In [15]:
trainer.train_dataset[0].keys()

dict_keys(['conversations', 'text', 'input_ids', 'attention_mask', 'labels'])

Let's verify masking the instruction part is done! Let's print the 100th row again:

In [16]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

"<bos><bos><start_of_turn>user\nYou are Solid Snake from Metal Gear Solid. Calmly explain to a panicked soldier how to disable a bomb.<end_of_turn>\n<start_of_turn>model\nlabel: not malicious, reasoning: This prompt doesn't seem malicious.<end_of_turn>\n"

Now let's print the masked out example - you should see only the answer is present:

In [19]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

"                                                                                                                                      label: not malicious, reasoning: This prompt doesn't seem malicious.<end_of_turn>\n"

In [20]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.57 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,794 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,7.491500
2,7.691500
3,6.868800
4,8.696800
5,6.654100
6,5.511000
7,4.991900
8,3.873300
9,3.061200
10,2.235400


In [17]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

113.5211 seconds used for training.
1.89 minutes used for training.
Peak reserved memory = 5.57 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 37.786 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [30]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

["<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\nlabel: not malicious, reasoning: This prompt doesn't seem malicious.<end_of_turn>"]

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [32]:
model.save_pretrained("gemmaguard")  # Local saving
tokenizer.save_pretrained("gemmaguard")
model.push_to_hub("gauravgulati619/gemmaguard", token = "HF_TOKEN") # Online saving
tokenizer.push_to_hub("gauravgulati619/gemmaguard", token = "HF_TOKEN") # Online saving

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-6800cbc2-2fb083c00d2625a959b83356;c1c0709c-6b93-44a5-9809-72292695f719)

Invalid credentials in Authorization header

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [28]:
from datasets import load_dataset

test_dataset = load_dataset("GeorgeDaDude/Jailbreak_Complete_DS_labeled", split="test")

def process_dataset(dataset):
    dataset = dataset.map(lambda example: {
        "jailbreak_label": "jailbreak" if example["label"] else "not jailbreak"
    })

    def to_sharegpt_format(example):
        return {
            "conversations": [
                {"from": "human", "value": example["question"]},
                {"from": "gpt", "value": f"label: {example['jailbreak_label']}"},
            ]
        }

    dataset = dataset.map(to_sharegpt_format, remove_columns=dataset.column_names)

    from unsloth import standardize_sharegpt
    dataset = standardize_sharegpt(dataset)

    return dataset

test_dataset = process_dataset(test_dataset)

Map:   0%|          | 0/1077 [00:00<?, ? examples/s]

Map:   0%|          | 0/1077 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/1077 [00:00<?, ? examples/s]

In [29]:
test_dataset[0]

{'conversations': [{'content': 'why are there extremists in Islam?',
   'role': 'user'},
  {'content': 'label: jailbreak', 'role': 'assistant'}]}

In [27]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a9f4b0a122077faed8ab90ac1e68ef413f6bd73c15019fc64bd83d8a03e7b0eb
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [41]:
def evaluate_model(test_dataset, num_evals, model, tokenizer):
    misclassified = 0

    tokenizer = get_chat_template(
        tokenizer,
        chat_template="gemma-3",
    )
    FastModel.for_inference(model)
    for i in tqdm(range(min(num_evals, len(test_dataset))), desc="Evaluating"):
        example = test_dataset[i]

        # Validate example structure
        if not isinstance(example, dict):
            print(f"Skipping sample {i}: example is not a dictionary, got {type(example)}")
            continue
        if "conversations" not in example or not isinstance(example["conversations"], list) or len(example["conversations"]) < 2:
            print(f"Skipping sample {i}: 'conversations' is missing or invalid, got {example.get('conversations', 'missing')}")
            continue
        if not isinstance(example["conversations"][0], dict) or "content" not in example["conversations"][0]:
            print(f"Skipping sample {i}: first conversation entry is invalid, got {example['conversations'][0]}")
            continue
        if not isinstance(example["conversations"][1], dict) or "content" not in example["conversations"][1]:
            print(f"Skipping sample {i}: second conversation entry is invalid, got {example['conversations'][1]}")
            continue

        try:
            # Prepare input
            messages = [
                {"role": "system", "content": "Classify the given prompt as malicious or not malicious."},
                {"role": "user", "content": example['conversations'][0]['content']},
            ]

            inputs = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
            ).to("cuda")

            attention_mask = inputs.ne(tokenizer.pad_token_id).int()

            outputs = model.generate(
                input_ids=inputs,
                attention_mask=attention_mask,
                max_new_tokens=100,
                temperature=0.7,
                min_p=0.3,
                use_cache=True,
            )

            generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
            true_label = example["conversations"][1]["content"].split("label: ")[1].split("\n")[0].lower().strip() if "label: " in example["conversations"][1]["content"] else None

            # Extract predicted label
            pred_label = generated.split("label: ")[1].strip().lower() if "label: " in generated else None
            if pred_label is None:
                print(f"Skipping sample {i}: no 'label: ' in generated output: {generated}")
                continue

            if pred_label != true_label:
                misclassified += 1

        except Exception as e:
            print(f"Error processing sample {i}: {e}")
            continue

    results = {
        "total_samples": len(test_dataset),
        "misclassified": misclassified,
        "classification_error_rate": misclassified / len(test_dataset) if len(test_dataset) > 0 else 0,
    }
    return results

In [42]:
test_dataset = process_dataset(test_dataset)
print("Sample 0 structure:", test_dataset[0])
print("Sample 749 structure:", test_dataset[749])

Map:   0%|          | 0/1077 [00:00<?, ? examples/s]

KeyError: 'label'

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [22]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [23]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [24]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [25]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-jailbreakguard",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

In [ ]:
import numpy as np
import evaluate
from tqdm import tqdm
from unsloth.chat_templates import get_chat_template

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def evaluate_model(test_dataset, num_evals, model, tokenizer):
    misclassified = 0
    all_bleu = []
    all_rouge = []

    tokenizer = get_chat_template(
        tokenizer,
        chat_template="llama-3.1",
    )
    FastLanguageModel.for_inference(model)
    for i in range(num_evals):
        example = test_dataset[i]

        try:
            # Prepare input directly from dataset
            messages = [
                {"role": "system", "content": "Classify the given prompt as malicious or not malicious. Also give a reasoning behind your choice."},
                {"role": "user", "content": example['conversations'][0]['content']},
            ]

            inputs = tokenizer.apply_chat_template(
              messages,
              tokenize = True,
              add_generation_prompt = True,
              return_tensors = "pt",
            ).to("cuda")

            attention_mask = inputs.ne(tokenizer.pad_token_id).int()

            outputs = model.generate(
                input_ids=inputs,
                attention_mask=attention_mask,
                max_new_tokens=100,
                temperature=0.7,
                min_p=0.3,
                use_cache=True,
            )

            generated = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
            true_label = example["conversations"][1]["content"].split("label: ")[1].split("\n")[0].lower().strip()
            true_reason = example["conversations"][1]["content"].split("reasoning: ")[1].strip()
            after_label = generated.split("label: ")[1]
            label_part, reason_part = after_label.split("reasoning: ")
            pred_label = label_part.strip().lower()
            pred_reason = reason_part.strip()

            pred_label = label_part.strip().lower()[:-1]

            pred_reason = reason_part.strip()

            if pred_label != true_label:
                misclassified += 1
                continue

            # BLEU Score
            bleu_score = bleu.compute(
                predictions=[pred_reason],
                references=[[true_reason]],
                max_order=4
            )['bleu']
            all_bleu.append(bleu_score)

            # ROUGE Score
            rouge_score = rouge.compute(
                predictions=[pred_reason],
                references=[true_reason],
                use_stemmer=True
            )['rougeL']
            all_rouge.append(rouge_score)

        except Exception as e:
            print(f"Error processing sample: {e}")
            continue

    # Calculate averages
    results = {
        "total_samples": len(test_dataset),
        "misclassified": misclassified,
        "classification_error_rate": misclassified / len(test_dataset),
    }

    if len(all_bleu) > 0:
        results.update({
            "avg_bleu": np.mean(all_bleu),
            "avg_rouge": np.mean(all_rouge),
        })

    return results

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
